In [1]:
### This script is for maf file processing -- Prepare multicore for patient with large mutations. May take >12hrs to run
### 1. MAF files are splitted into individual files
### 2. Mutation categories are assigned to individuals
### 3. Individual files are merged into histology files

In [1]:
from functools import partial
from maf_utils import*

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
### Define maf
class maf_process:
    def __init__(self, params):
        self.dir_maf_split = params['dir_maf_split']
        self.dir_out = params['dir_out']
        self.dir_out_intermediate = params['dir_out_intermediate']
        self.dir_out_intermediate_ind_split = os.path.join(params['dir_out_intermediate'],'individual/split/')
        self.dir_out_intermediate_ind_categ = os.path.join(params['dir_out_intermediate'],'individual/categ/')
        self.ncore = params['parallelize_core']
        
        ### reference folder and files
        self.dir_ref = '../data/proc_refs/'
        self.fname_patient_list = 'list_all_patients_102121.pkl'
        self.histology_dfname = 'histology.csv'
        self.histology_nohype_dfname = 'histology_nohypermutator.csv'
        self.fname_lgene_old = 'gene_name_list_062121.pkl' # this is gene name list only, the old one has 19225 genes 
        self.fname_lgene_new = 'gene_name_list_102121.pkl' # the new one has 18000ish genes
        ### output folder and intermediate folder
        if not os.path.exists(self.dir_out):
            os.makedirs(self.dir_out)
        if not os.path.exists(self.dir_out_intermediate):
            os.makedirs(self.dir_out_intermediate)
        if not os.path.exists(self.dir_out_intermediate_ind_split):
            os.makedirs(self.dir_out_intermediate_ind_split)
        if not os.path.exists(self.dir_out_intermediate_ind_categ):
            os.makedirs(self.dir_out_intermediate_ind_categ)
            
    ###-------------------------
    # 1. First create intermediate file individual patient maf files
    ###-------------------------    
    def run_split_patient(self): # not using multiprocessing...afrain file read/write will confilct
        # get the list of splitted maf files
        lmaf_split = os.listdir(self.dir_maf_split)
        for maf in tqdm(lmaf_split):
            split_patient(maf,dir_maf = self.dir_maf_split, dir_out = self.dir_out_intermediate_ind_split)
        print('Finish making individual patient maf files...')
    
    ###-------------------------
    # 2. Assign mutation category to individual patients
    ###-------------------------    
    def run_assign_categ(self):
        ### Get the all patient lists
        self.patient_list = pickle.load(open(os.path.join(self.dir_ref,self.fname_patient_list), 'rb'))
        print('Start assigning categ to individual patients...')
        processes = []
        
        ### Multiprocessing
        # for patients in self.patient_list:
        #     p = mp.Process()
        #     function1 = partial(categ_assign,dir_ind = self.dir_out_intermediate_ind_split,\
        #                    dir_categ_out = self.dir_out_intermediate_ind_categ)
        #     p = mp.Process(target=function1, args=(patients,))
        #     processes.append(p)
        # [x.start() for x in processes]
        # [x.join() for x in processes]
        # [x.close() for x in processes]

        self.runcateg = pickle.load(open('lruncateg.pkl','rb'))

        ## Not multiprocessing
        for patients in tqdm(self.runcateg):
            categ_assign(patients,dir_ind = self.dir_out_intermediate_ind_split,\
                           dir_categ_out = self.dir_out_intermediate_ind_categ )

        print('Finish assigning categ to individual patients...')
    
    ###-------------------------
    # 3. Merge individual mutation file to histology mutation files
    ###-------------------------    
    def load_histology_info(self, gene_name = 'old'):
        print('Loading histology cohort & gene reference data')
        self.histology_df = pd.read_csv(os.path.join(self.dir_ref,self.histology_dfname))
        self.histology_nohype_df = pd.read_csv(os.path.join(self.dir_ref,self.histology_nohype_dfname))
        # if use the old gene names
        if gene_name == 'old':
            self.gene_name_list = pickle.load(open(os.path.join(self.dir_ref, self.fname_lgene_old),'rb'))
        elif gene_name == 'new':
            self.gene_name_list = pickle.load(open(os.path.join(self.dir_ref, self.fname_lgene_new),'rb'))
        print('Finish Loading histology cohort & gene reference data')
    
    # merge patients
    def merge_maf(self, feature, hypermutator = False, filter_gene = False):
        if hypermutator:
            df_sample = self.histology_nohype_df
            self.dir_out_merged = self.dir_out+'_nohypermutator'
            if not os.path.exists(self.dir_out_merged):
                os.makedirs(self.dir_out_merged)
        else: 
            df_sample = self.histology_df
            self.dir_out_merged = self.dir_out

        if os.path.exists(os.path.join(self.dir_out_merged,feature+'.csv.gz')):
            print(f'exists{feature}')
            return

        # Initialize list for append
        ldf = []

        # Get patient list for histology
        df_histology = df_sample[df_sample['histology'] == feature]
        lp = df_histology['tumor_aliquot_id'].unique()

        #read patient file
        for p in tqdm(lp):
            df_maf_ind = pd.read_csv(os.path.join(self.dir_out_intermediate_ind_categ,p+'.to_merge.categ.csv'), index_col = 0)
            if filter_gene:
                df_maf_ind = df_maf_ind[df_maf_ind['Hugo_Symbol'].isin(self.gene_name_list)] ## Filter genes
                ldf.append(df_maf_ind)
            else:
                ldf.append(df_maf_ind)

        df_maf_merged = pd.concat(ldf, axis = 0)
        df_maf_merged['categ'] = df_maf_merged['categ'].astype(int)
        df_maf_merged.to_csv(os.path.join(self.dir_out_merged,feature+'.csv.gz'), sep = '\t', index = False\
         ,chunksize=100000,compression='gzip',encoding='utf-8')
        
        print(f'Finish Merging...')

In [11]:
maf_params = {'dir_maf_split': '../maf_raw/maf_split',
              'dir_out_intermediate': '../data/maf/intermediate',
              'dir_out': '../data/maf/histology',
              'parallelize_core':6} 

In [ ]:
### Split patient file, don't run if you already run
res = maf_process(maf_params)
res.run_split_patient()

In [ ]:
### Assign categ to individual patient files, don't run if you already run
res = maf_process(maf_params)
res.run_assign_categ()

In [5]:
### Merge patient individual mutations into a cohort
maf_params = {'dir_maf_split': '../maf_raw/maf_split',
              'dir_out_intermediate': '../data/maf/intermediate',
              'dir_out': '../data/maf/histology_all',
              'parallelize_core':6} 
df_cohort = pd.read_csv(os.path.join('../data/proc_refs/histology.csv'))
lfeat = df_cohort['histology'].unique()
print(lfeat)

res = maf_process(maf_params)
res.load_histology_info(gene_name = 'new')

for histologies in lfeat:
    res.merge_maf(histologies, hypermutator = False, filter_gene = True)
# for histologies in lfeat:
#     res.merge_maf(histologies, True)

  2%|▏         | 2/109 [00:00<00:07, 15.03it/s]

['Ovary-AdenoCA' 'CNS-PiloAstro' 'Liver-HCC' 'CNS-Oligo' 'Panc-Endocrine'
 'Kidney-RCC' 'Prost-AdenoCA' 'Thy-AdenoCA' 'ColoRect-AdenoCA'
 'Lymph-BNHL' 'Uterus-AdenoCA' 'Breast-AdenoCA' 'Lung-AdenoCA'
 'Panc-AdenoCA' 'Eso-AdenoCA' 'Head-SCC' 'CNS-Medullo' 'CNS-GBM'
 'SoftTissue-Leiomyo' 'Cervix-SCC' 'Skin-Melanoma' 'Lymph-CLL'
 'SoftTissue-Liposarc' 'Kidney-ChRCC' 'Stomach-AdenoCA' 'Lung-SCC'
 'Bladder-TCC' 'Myeloid-AML' 'Biliary-AdenoCA' 'Breast-LobularCA'
 'Cervix-AdenoCA' 'Bone-Osteosarc' 'Breast-DCIS' 'Myeloid-MPN'
 'Myeloid-MDS' 'Bone-Cart' 'Bone-Osteoblast' 'Bone-Epith' 'Bone-Benign']
Loading histology cohort & gene reference data
Finish Loading histology cohort & gene reference data


  4%|▍         | 4/89 [00:00<00:02, 34.49it/s]

Finish Merging...


  0%|          | 1/312 [00:00<00:45,  6.77it/s]

...

Finish Merging...
Finish Merging...


100%|██████████| 5/5 [00:00<00:00, 29.48it/s]

Finish Merging...
Finish Merging...



100%|██████████| 1/1 [00:00<00:00, 52.32it/s]

Finish Merging...
Finish Merging...


***Directly split maf file from icgc and tcga maf files***

In [1]:
import os,pickle
import pandas as pd
from tqdm import tqdm

In [2]:
dir_maf = '../maf_raw/'

In [ ]:
dficgc = pd.read_csv(os.path.join(dir_maf, 'icgc.maf'), sep = '\t')

In [6]:
maf_split = [pd.DataFrame(y) for x, y in dficgc.groupby('Tumor_Sample_Barcode', as_index=False)]
for df_split in tqdm(maf_split):
    patient = df_split['Tumor_Sample_Barcode'].unique().tolist()
    if len(patient) == 1:
        # df_split.to_csv(os.path.join('../data/maf/intermediate/individual/split', patient[0]+'.to_merge.csv'))
    else:
        print(f'ERROR{patient}')

100%|██████████| 1950/1950 [06:20<00:00,  5.13it/s]


In [3]:
dftcga = pd.read_csv(os.path.join(dir_maf, 'tcga.maf'), sep = '\t')

/storage/home/yur97/anaconda3/envs/mSigsyn/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,17,24,28,33,34,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
maf_split = [pd.DataFrame(y) for x, y in dftcga.groupby('Tumor_Sample_Barcode', as_index=False)]
for df_split in tqdm(maf_split):
    patient = df_split['Tumor_Sample_Barcode'].unique().tolist()
    if len(patient) == 1:
        # df_split.to_csv(os.path.join('../data/maf/intermediate/individual/split', patient[0]+'.to_merge.csv'))
    else:
        print(f'ERROR{patient}')

100%|██████████| 828/828 [08:29<00:00,  1.63it/s]  
